In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import svm
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
import math
import datetime
import multiprocessing as mp
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import random


# In[25]:


# cores = mp.cpu_count()
# pool = mp.Pool(processes=20)

# min1=min(list(X_standard.shape))
# a1=list(range(10,min1,10))
# a1[-1]=min1
# pcc_top1=pool.map(find_best_pca_lr,a1)  ##（SVM，LR）
# pool.terminate()


# a1=pd.DataFrame()
# for i1 in pcc_top1:
#     a1=pd.concat([a1,i1], axis=0)
# a1.columns=['tr_TP', 'tr_TN', 'tr_FP', 'tr_FN', 'tr_Sen', 'tr_Spe', 'tr_Acc', 'tr_PPV', 'tr_NPV', 'tr_MCC', 'tr_AUC','TP',
#        'TN', 'FP', 'FN', 'Sen', 'Spe', 'Acc', 'PPV', 'NPV', 'MCC', 'AUC','PCA']   

# a1=a1.sort_values(['Acc','tr_Acc','PCA'], ascending=[False,False,True])

In [2]:
def read_single_csv(input_path,i):

    df_chunk=pd.read_csv(input_path,
                         chunksize=10000,
#                          sep='\t'
                        sep=i
                        )
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [3]:
def get_measures_gridloo(label, score):
    label = np.array(label)
    score = np.array(score)
    
    N  = len(label)
    TP = sum((label == 1) & (score == 1))
    TN = sum((label == 0) & (score == 0))
    FP = sum((label == 0) & (score == 1))
    FN = sum((label == 1) & (score == 0))

    # init all measures to nan
    measures = {measure: float("nan") for measure in ("Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC")}
    
    measures["TP"] = TP
    measures["TN"] = TN
    measures["FP"] = FP
    measures["FN"] = FN
    
    S = (TP + FN) / N
    P = (TP + FP) / N

    if (TP + FN) > 0:
        measures["Sen"] = round(TP/(TP+FN), 4)

    if (TN + FP) > 0:
        measures["Spe"] = round(TN/(TN+FP), 4)

    if (TP + FP + FN + TN) > 0:
        measures["Acc"] = round((TP+TN)/(TP+FP+FN+TN), 4)

    if (TP + FP) > 0:
        measures["PPV"] = round(TP/(TP+FP), 4)

    if (TN + FN) > 0:
        measures["NPV"] = round(TN/(TN+FN), 4)

    if (S*P*(1-S)*(1-P)) > 0:
        measures["MCC"] = round((TP/N - S*P)/(math.sqrt(S*P*(1-S)*(1-P))), 4)
    
    
    measures["AUC"]= roc_auc_score(label, score)
    return pd.DataFrame([measures],
                        columns=["TP", "TN", "FP",
                                 "FN", "Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC"])

In [4]:
##SVM
def find_best_pca(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
    grid_search.fit(pca_X_train, train_y)
    
    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [5]:
def find_best_pca_lr(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

    param_grid = {'penalty':['l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],  
              "solver":["newton-cg", "lbfgs","liblinear"],
#             "C":list(np.arange(0,0.1,0.005)),   
#               "solver":["newton-cg", "lbfgs","liblinear"]
            }
    LR_grid = LogisticRegression()
    grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=9)
    grid_search.fit(pca_X_train, train_y)


    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
   

    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [101]:
cancer='BRCA'
path='D:\\lymph_meta_xmiseq\\tcga_data\\'
# e1="./"+cancer+"_delta_pcc_wilcox.csv"
# delta_pcc=read_single_csv(e1,',')
# delta_pcc=delta_pcc.drop(['Unnamed: 0','pair'],axis=1)
# delta_pcc=delta_pcc.dropna(axis=0,how='any')

# result=delta_pcc.T
# train=result.iloc[:,0:-1].values.astype('float')
# target=result.iloc[:,-1].values.astype('float')

train=pd.read_csv(path+cancer+"\\admat_final.csv").iloc[:,2:].T.values
target=pd.read_csv(path+cancer+"\\gctm_label.csv",index_col=0).values.ravel()

In [86]:
train

array([[ 0.00874834, -0.00736022, -0.01710605, ...,  0.0059139 ,
         0.00585509,  0.01760065],
       [-0.06756009, -0.03635654, -0.06104614, ...,  0.00184041,
         0.01073223, -0.00660949],
       [ 0.05017487, -0.03385318, -0.05728117, ..., -0.01067972,
         0.01398897,  0.01295414],
       ...,
       [-0.00012984,  0.00158047,  0.00504004, ...,  0.00526114,
        -0.00421565, -0.0168382 ],
       [-0.0120549 ,  0.0223974 , -0.02588812, ..., -0.03524445,
         0.00812885,  0.01039054],
       [ 0.00757404,  0.00548185,  0.02304116, ...,  0.00104388,
         0.00138618, -0.0349668 ]])

In [87]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [183]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)

n_components=0.95
estimator = PCA(n_components=n_components,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)
pca_X_test = estimator.transform(X_standard_test)
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

In [184]:
train_X.shape

(126, 2281)

In [185]:
#svm
scoring = {'AUC': 'roc_auc'}
cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
svc_grid_search.fit(pca_X_train, train_y)

#LR
param_grid = {'penalty':['l1', 'l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
#               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
            }
LR_grid = LogisticRegression(max_iter=1000)
LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
LR_grid_search.fit(pca_X_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.45230769        nan 0.52369231 0.54769231
 0.54769231 0.72153846 0.76123077 0.74523077        nan        nan
 0.45230769        nan 0.50030769 0.75292308 0.75292308 0.75292308
 0.79230769 0.75292308        nan        nan 0.45230769        nan
 0.52369231 0.848      0.848      0.82430769 0.864      0.832
        nan        nan 0.68246154        nan 0.75292308 0.888
 0.888      0.87230769 0.88       0.88              nan        nan
 0.92              nan 0.92       0.88061538 0.88061538 0.85661538
 0.86461538 0.86461538        nan        nan 0.944             nan
 0.936      0.84061538 0.84061538 0.86461538 0.86461538 0.87261538
        nan        nan 0.944             nan 0.87261538 0.84861538
 0.84861538 0.86461538 0.86461538 0.87261538        nan        nan
 0.90430769        nan 0.87261538 0.85661538 0.85661538 0.84861

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=10,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [186]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]
clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
clf.fit(pca_X_train, train_y)

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LinearSVC(C=5), n_jobs=10)

In [187]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]

param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                 LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                 LogisticRegression(C=1000)]}

Stacking_grid =StackingClassifier(estimators=estimators,)
Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
Stacking_grid_search.fit(pca_X_train, train_y)
Stacking_grid_search.best_estimator_

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LogisticRegression(C=100))

In [188]:
train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components

test_pre_y = Stacking_grid_search.predict(pca_X_test)
test_res1=get_measures_gridloo(test_y,test_pre_y)

In [189]:
print(train_res1)
print(test_res1)

   TP  TN  FP  FN     Sen     Spe     Acc     PPV     NPV     MCC       AUC
0  64  50   7   5  0.9275  0.8772  0.9048  0.9014  0.9091  0.8076  0.902365
   TP  TN  FP  FN  Sen  Spe  Acc  PPV  NPV  MCC  AUC
0  30  24   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0


In [104]:
%%time
################random seed loop#################
train_scores=[]
test_scores=[]
loopn=20
scoring='roc_auc'
np.random.seed(42)
random_states=np.random.choice(range(101), loopn, replace=False)

tmp_train=[]
tmp_test=[]
#print(random_states)
for i in range(loopn):
    train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_states[i]
                                                   )
    print("train_x.shpae:")
    print(train_X.shape)

    standardScaler = StandardScaler()
    standardScaler.fit(train_X)
    X_standard = standardScaler.transform(train_X)
    X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
    estimator = PCA(n_components=0.99,random_state=random_states[i])
    pca_X_train = estimator.fit_transform(X_standard)

    n_components=range(10,min(pca_X_train.shape),10)
    print(n_components)
    #n_components=[0.99,0.95,0.90,0.85]
    best_pca_train_scores=[]
    best_pca_test_scores=[]
    for j in n_components:
        estimator = PCA(n_components=j,random_state=random_states[i])
        pca_X_train = estimator.fit_transform(X_standard)
        pca_X_test = estimator.transform(X_standard_test)
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i]) 
    
        #scoring = {'AUC': 'roc_auc'}
        cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
        gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
        parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

        svc_grid_search=GridSearchCV(estimator=SVC(random_state=random_states[i]),
                                     param_grid=parameters,cv=cvx,scoring=scoring)
        svc_grid_search.fit(pca_X_train, train_y)

        param_grid = {'penalty':['l1', 'l2'],
                      "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                      "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                    }
        LR_grid = LogisticRegression(max_iter=1000, random_state=random_states[i])
        LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        LR_grid_search.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]
        clf = StackingClassifier(estimators=estimators, 
                                 final_estimator=LinearSVC(C=5, random_state=random_states[i]),n_jobs=10)
        clf.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]

        param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                         LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                         LogisticRegression(C=0.1),LogisticRegression(C=1),
                                         LogisticRegression(C=10),LogisticRegression(C=100),
                                         LogisticRegression(C=1000)]}

        Stacking_grid =StackingClassifier(estimators=estimators,)
        Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                            scoring=scoring,n_jobs=10)
        Stacking_grid_search.fit(pca_X_train, train_y)
        Stacking_grid_search.best_estimator_

        train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
        train_res1=get_measures_gridloo(train_y,train_pre_y)

        test_pre_y = Stacking_grid_search.predict(pca_X_test)
        test_res1=get_measures_gridloo(test_y,test_pre_y)
        
        best_pca_train_scores.append(train_res1.loc[:,"AUC"])
        best_pca_test_scores.append(test_res1.loc[:,"AUC"])
    
    train_scores.append(np.max(best_pca_train_scores))
    test_scores.append(best_pca_test_scores[np.argmax(best_pca_train_scores)])
    print("n_components:")
    print(n_components[np.argmax(best_pca_train_scores)])
    
    tmp_train.append(best_pca_train_scores)
    tmp_test.append(best_pca_test_scores)

train_x.shpae:
(643, 4660)
range(10, 487, 10)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.70783608
 0.70783608 0.70846579 0.70841769 0.70846579        nan        nan
 0.5               nan 0.5        0.71177256 0.7118214  0.70934767
 0.70929883 0.70925073        nan        nan 0.65761794        nan
 0.65761794 0.7430125  0.7430125  0.72077686 0.72460371 0.72058085
        nan        nan 0.74859391        nan 0.75014418 0.78432475
 0.78432475 0.74467403 0.73256734 0.72626864        nan        nan
 0.77537518        nan 0.72930509 0.79748319 0.79748319 0.78308104
 0.73324446 0.72728168        nan        nan 0.79748261        nan
 0.72752509 0.7988921  0.7988921  0.79748471 0.73333992 0.72723288
        nan        nan 0.79893946        nan 0.72728022 0.79893951
 0.79893951 0.79889215 0.73329182 0.72723288        nan        nan
 0.79893951        nan 0.72723288 0.79893798 0.798937

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.72913418
 0.72913418 0.72937456 0.72932648 0.72932648        nan        nan
 0.5               nan 0.5        0.73592514 0.73592514 0.73481229
 0.73486039 0.73490773        nan        nan 0.65761794        nan
 0.65761794 0.76685072 0.7668988  0.75069819 0.75263456 0.75007023
        nan        nan 0.74795614        nan 0.74999482 0.80721737
 0.80726621 0.76642492 0.75718365 0.75371635        nan        nan
 0.80268738        nan 0.76045288 0.81159669 0.81159669 0.79365503
 0.75587297 0.75342359        nan        nan 0.80901265        nan
 0.75444181 0.80532801 0.80527991 0.80504003 0.75563106 0.75366552
        nan        nan 0.80473816        nan 0.75347393 0.8048862
 0.80478928 0.80459543 0.75563106 0.75371436        nan        nan
 0.80473968        nan 0.7537632  0.80478778 0.8047877

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.7328549
 0.7328549  0.73353038 0.73353114 0.73353114        nan        nan
 0.5               nan 0.5        0.7438357  0.7438357  0.74300692
 0.74295811 0.74295811        nan        nan 0.65761794        nan
 0.65761794 0.77767686 0.77762876 0.76263045 0.76403189 0.7626289
        nan        nan 0.74761796        nan 0.75023754 0.79984485
 0.79984485 0.76515725 0.76703338 0.76606128        nan        nan
 0.79834647        nan 0.77562616 0.79285037 0.79285037 0.76317116
 0.76567872 0.76571505        nan        nan 0.78791768        nan
 0.76716903 0.77876038 0.77876038 0.772787   0.76543598 0.76576239
        nan        nan 0.77537555        nan 0.7659074  0.77362765
 0.77362765 0.77387266 0.76533904 0.76571355        nan        nan
 0.77280851        nan 0.76576313 0.7729535  0.7729535 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73454788
 0.73454788 0.73469438 0.73469438 0.73469438        nan        nan
 0.5               nan 0.5        0.74740817 0.74740817 0.74619246
 0.74590315 0.74600083        nan        nan 0.65761794        nan
 0.65761794 0.78504768 0.78504768 0.7712278  0.77219336 0.77093041
        nan        nan 0.74761796        nan 0.75023754 0.79174899
 0.79174899 0.76638667 0.7747777  0.77656406        nan        nan
 0.77258008        nan 0.77341786 0.76045179 0.76040295 0.73838798
 0.77404519 0.77569329        nan        nan 0.75234411        nan
 0.77665839 0.73358225 0.73358225 0.71526619 0.77331779 0.77593531
        nan        nan 0.71436471        nan 0.77583911 0.71388445
 0.71388519 0.71045653 0.77341399 0.77593531        nan        nan
 0.70087572        nan 0.77588724 0.70208269 0.702082

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.72991989
 0.72991989 0.7300672  0.7300672  0.73011528        nan        nan
 0.5               nan 0.5        0.73313811 0.73313811 0.73182537
 0.73177875 0.73158265        nan        nan 0.71323343        nan
 0.71323343 0.75909308 0.75909308 0.74277093 0.74616742 0.74369679
        nan        nan 0.72849879        nan 0.734441   0.75496589
 0.75496589 0.71179994 0.72566891 0.73368025        nan        nan
 0.74866011        nan 0.73239023 0.72420959 0.72416149 0.67510705
 0.7219876  0.73160319        nan        nan 0.71615752        nan
 0.73276332 0.70040945 0.70050713 0.66612552 0.72189218 0.73150551
        nan        nan 0.66917735        nan 0.73140859 0.67343753
 0.67338943 0.66092354 0.72189218 0.73145667        nan        nan
 0.65578757        nan 0.73135975 0.66556885 0.665520

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.7319653
 0.7319653  0.73225682 0.7321599  0.73230566        nan        nan
 0.5               nan 0.5        0.73658487 0.73658487 0.73489041
 0.73489041 0.73503545        nan        nan 0.71323343        nan
 0.71323343 0.76893414 0.76893414 0.75242947 0.75499778 0.75247523
        nan        nan 0.72844995        nan 0.734441   0.76986509
 0.76991317 0.73106068 0.74543907 0.75070622        nan        nan
 0.75142647        nan 0.73852631 0.738873   0.73892108 0.69469537
 0.74099149 0.74963534        nan        nan 0.71825466        nan
 0.74756228 0.70140963 0.70140963 0.66128857 0.74046019 0.7494437
        nan        nan 0.67717491        nan 0.74963761 0.68681626
 0.68676742 0.64851505 0.74046019 0.74959022        nan        nan
 0.65759904        nan 0.74954138 0.68286628 0.68286628

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73084934
 0.73084934 0.73118967 0.73118967 0.73114157        nan        nan
 0.5               nan 0.5        0.73604893 0.73604893 0.73459945
 0.73430791 0.73411406        nan        nan 0.71323343        nan
 0.71323343 0.76888537 0.76888537 0.75247021 0.75624739 0.75338617
        nan        nan 0.72854611        nan 0.734441   0.7710801
 0.771032   0.73330089 0.74426636 0.74874783        nan        nan
 0.72958322        nan 0.73348917 0.73295751 0.73295751 0.69190025
 0.74132883 0.74769217        nan        nan 0.71447687        nan
 0.74639247 0.70479157 0.70474273 0.66139458 0.74137841 0.74749681
        nan        nan 0.69056728        nan 0.74735401 0.69673643
 0.69678453 0.64908853 0.74123411 0.74749681        nan        nan
 0.65173606        nan 0.74744797 0.69126013 0.6912586

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73065257
 0.73065257 0.73089378 0.73089378 0.73094186        nan        nan
 0.5               nan 0.5        0.73648583 0.73648583 0.73502819
 0.73498083 0.73488391        nan        nan 0.71323343        nan
 0.71323343 0.77163288 0.77163288 0.75491874 0.75855575 0.7558859
        nan        nan 0.72844995        nan 0.734441   0.78098582
 0.78098582 0.74683728 0.75170038 0.75352472        nan        nan
 0.72606775        nan 0.73225224 0.74567386 0.74567386 0.71184577
 0.74789354 0.75231879        nan        nan 0.70723496        nan
 0.74987119 0.71641694 0.71646578 0.68278444 0.74765239 0.75227148
        nan        nan 0.68088624        nan 0.7523699  0.70741268
 0.70741268 0.67089281 0.74770045 0.75227148        nan        nan
 0.64081504        nan 0.75222416 0.70198216 0.7018371

n_components:
10
Wall time: 15h 7min 28s


In [105]:
print('mean')
print(np.mean(train_scores))
print(np.mean(test_scores))
print("std")
print(np.std(train_scores))
print(np.std(test_scores))
pd.DataFrame(tmp_train)

mean
0.7493835404327798
0.7397321428571428
std
0.012532381791542108
0.022023953907401497


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,"0 0.777194 Name: AUC, dtype: float64","0 0.76217 Name: AUC, dtype: float64","0 0.770453 Name: AUC, dtype: float64","0 0.774934 Name: AUC, dtype: float64","0 0.767129 Name: AUC, dtype: float64","0 0.75584 Name: AUC, dtype: float64","0 0.737066 Name: AUC, dtype: float64","0 0.752729 Name: AUC, dtype: float64","0 0.75136 Name: AUC, dtype: float64","0 0.724993 Name: AUC, dtype: float64",...,"0 0.712495 Name: AUC, dtype: float64","0 0.726096 Name: AUC, dtype: float64","0 0.709064 Name: AUC, dtype: float64","0 0.730737 Name: AUC, dtype: float64","0 0.727732 Name: AUC, dtype: float64","0 0.737013 Name: AUC, dtype: float64","0 0.733848 Name: AUC, dtype: float64",None,None,None
1,"0 0.743342 Name: AUC, dtype: float64","0 0.748142 Name: AUC, dtype: float64","0 0.738808 Name: AUC, dtype: float64","0 0.737226 Name: AUC, dtype: float64","0 0.73543 Name: AUC, dtype: float64","0 0.743289 Name: AUC, dtype: float64","0 0.758899 Name: AUC, dtype: float64","0 0.719501 Name: AUC, dtype: float64","0 0.719767 Name: AUC, dtype: float64","0 0.716762 Name: AUC, dtype: float64",...,"0 0.703785 Name: AUC, dtype: float64","0 0.707003 Name: AUC, dtype: float64","0 0.705367 Name: AUC, dtype: float64","0 0.713066 Name: AUC, dtype: float64","0 0.703838 Name: AUC, dtype: float64","0 0.702203 Name: AUC, dtype: float64",None,None,None,None
2,"0 0.732585 Name: AUC, dtype: float64","0 0.746826 Name: AUC, dtype: float64","0 0.72615 Name: AUC, dtype: float64","0 0.749777 Name: AUC, dtype: float64","0 0.743395 Name: AUC, dtype: float64","0 0.758952 Name: AUC, dtype: float64","0 0.737385 Name: AUC, dtype: float64","0 0.730843 Name: AUC, dtype: float64","0 0.738755 Name: AUC, dtype: float64","0 0.715233 Name: AUC, dtype: float64",...,"0 0.733795 Name: AUC, dtype: float64","0 0.727679 Name: AUC, dtype: float64","0 0.730737 Name: AUC, dtype: float64",None,None,None,None,None,None,None
3,"0 0.724621 Name: AUC, dtype: float64","0 0.699517 Name: AUC, dtype: float64","0 0.707375 Name: AUC, dtype: float64","0 0.738701 Name: AUC, dtype: float64","0 0.729261 Name: AUC, dtype: float64","0 0.721403 Name: AUC, dtype: float64","0 0.725937 Name: AUC, dtype: float64","0 0.722879 Name: AUC, dtype: float64","0 0.705474 Name: AUC, dtype: float64","0 0.70542 Name: AUC, dtype: float64",...,"0 0.692656 Name: AUC, dtype: float64","0 0.689651 Name: AUC, dtype: float64","0 0.67573 Name: AUC, dtype: float64",None,None,None,None,None,None,None
4,"0 0.743182 Name: AUC, dtype: float64","0 0.741493 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.759005 Name: AUC, dtype: float64","0 0.743022 Name: AUC, dtype: float64","0 0.7464 Name: AUC, dtype: float64","0 0.744818 Name: AUC, dtype: float64","0 0.746294 Name: AUC, dtype: float64","0 0.729155 Name: AUC, dtype: float64","0 0.735643 Name: AUC, dtype: float64",...,"0 0.712746 Name: AUC, dtype: float64","0 0.708851 Name: AUC, dtype: float64","0 0.707216 Name: AUC, dtype: float64","0 0.699198 Name: AUC, dtype: float64","0 0.711962 Name: AUC, dtype: float64","0 0.705474 Name: AUC, dtype: float64",None,None,None,None
5,"0 0.57817 Name: AUC, dtype: float64","0 0.568251 Name: AUC, dtype: float64","0 0.747823 Name: AUC, dtype: float64","0 0.727306 Name: AUC, dtype: float64","0 0.724301 Name: AUC, dtype: float64","0 0.731893 Name: AUC, dtype: float64","0 0.716337 Name: AUC, dtype: float64","0 0.732159 Name: AUC, dtype: float64","0 0.711856 Name: AUC, dtype: float64","0 0.728835 Name: AUC, dtype: float64",...,"0 0.740177 Name: AUC, dtype: float64","0 0.721296 Name: AUC, dtype: float64","0 0.730684 Name: AUC, dtype: float64","0 0.73696 Name: AUC, dtype: float64","0 0.73063 Name: AUC, dtype: float64","0 0.739964 Name: AUC, dtype: float64",None,None,None,None
6,"0 0.739964 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.708638 Name: AUC, dtype: float64","0 0.71054 Name: AUC, dtype: float64","0 0.727626 Name: AUC, dtype: float64","0 0.723145 Name: AUC, dtype: float64","0

In [106]:
print(train_scores)
print(test_scores)

[0.9295183982683983]
[0    0.940972
Name: AUC, dtype: float64]


In [102]:
random_stat=10
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_stat
                                                   )
print("train_x.shpae:")
print(train_X.shape)

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
estimator = PCA(n_components=0.99,random_state=random_stat)
pca_X_train = estimator.fit_transform(X_standard)

n_components=range(10,min(pca_X_train.shape),10)
print(n_components)
    #n_components=[0.99,0.95,0.90,0.85]
best_pca_train_scores=[]
best_pca_test_scores=[]
for j in n_components:
    estimator = PCA(n_components=j,random_state=random_stat)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_stat) 
    
        #scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

    svc_grid_search=GridSearchCV(estimator=SVC(random_state=random_stat),
                                 param_grid=parameters,cv=cvx,scoring=scoring)
    svc_grid_search.fit(pca_X_train, train_y)

    param_grid = {'penalty':['l1', 'l2'],
                  "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                  }
    LR_grid = LogisticRegression(max_iter=1000, random_state=random_stat)
    LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
    LR_grid_search.fit(pca_X_train, train_y)

    estimators = [
        ('lr', LR_grid_search.best_estimator_),
        ('svc', svc_grid_search.best_estimator_),
    ]
    clf = StackingClassifier(estimators=estimators, 
                             final_estimator=LinearSVC(C=5, random_state=random_stat),n_jobs=10)
    clf.fit(pca_X_train, train_y)

    estimators = [
        ('lr', LR_grid_search.best_estimator_),
        ('svc', svc_grid_search.best_estimator_),
    ]

    param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                    LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                    LogisticRegression(C=0.1),LogisticRegression(C=1),
                                    LogisticRegression(C=10),LogisticRegression(C=100),
                                    LogisticRegression(C=1000)]}

    Stacking_grid =StackingClassifier(estimators=estimators,)
    Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                        scoring=scoring,n_jobs=10)
    Stacking_grid_search.fit(pca_X_train, train_y)
    Stacking_grid_search.best_estimator_

    train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)

    test_pre_y = Stacking_grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
        
    best_pca_train_scores.append(train_res1.loc[:,"AUC"])
    best_pca_test_scores.append(test_res1.loc[:,"AUC"])

train_x.shpae:
(643, 4660)
range(10, 463, 10)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.7143901
 0.7143901  0.71497091 0.71497091 0.71497091        nan        nan
 0.5               nan 0.5        0.71448764 0.71448764 0.71269409
 0.71288869 0.71283985        nan        nan 0.75544363        nan
 0.75544363 0.7410049  0.7410049  0.71928803 0.72452302 0.72074056
        nan        nan 0.72603681        nan 0.72980901 0.77485496
 0.77485496 0.73512124 0.72803268 0.72116555        nan        nan
 0.76355163        nan 0.72089623 0.78226702 0.78226702 0.76229474
 0.72811836 0.72164791        nan        nan 0.78163126        nan
 0.72198905 0.78337635 0.78337635 0.78181982 0.7286038  0.72184327
        nan        nan 0.78313284        nan 0.72184327 0.78327786
 0.78322902 0.78332596 0.72865188 0.72184327        nan        nan
 0.78318092        nan 0.72184327 0.78327709 0.7832770

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73633871
 0.73633871 0.73653178 0.73653104 0.73653104        nan        nan
 0.5               nan 0.5        0.74269416 0.74269416 0.74138156
 0.74118694 0.74113958        nan        nan 0.75544363        nan
 0.75544363 0.76983163 0.76983163 0.75093999 0.75438573 0.75225268
        nan        nan 0.73228325        nan 0.73698679 0.79197839
 0.79197839 0.74041255 0.74103688 0.74431337        nan        nan
 0.78788249        nan 0.74149991 0.8082046  0.8082046  0.75645829
 0.73864691 0.74246402        nan        nan 0.80935527        nan
 0.74217026 0.80649448 0.80649448 0.79157322 0.73854918 0.7421251
        nan        nan 0.80668741        nan 0.74236557 0.80649348
 0.80649348 0.80464598 0.7384515  0.7421732         nan        nan
 0.80644309        nan 0.7421251  0.80639647 0.8063957

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73808821
 0.73808821 0.73866899 0.73871783 0.73866899        nan        nan
 0.5               nan 0.5        0.74564311 0.74564311 0.7441435
 0.74424041 0.74423967        nan        nan 0.75544363        nan
 0.75544363 0.77876228 0.77876228 0.76098621 0.76446891 0.76214864
        nan        nan 0.73233063        nan 0.73708373 0.78899101
 0.78899101 0.7413675  0.74847613 0.75538843        nan        nan
 0.78168824        nan 0.7591758  0.78937373 0.78937373 0.7388036
 0.74463534 0.75369464        nan        nan 0.78202332        nan
 0.75404336 0.76297335 0.76297335 0.7466749  0.74385314 0.75364427
        nan        nan 0.75466184        nan 0.7533557  0.75180572
 0.75180572 0.75025777 0.74390198 0.75374195        nan        nan
 0.74976415        nan 0.75369311 0.74952221 0.74952221

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73972007
 0.73967197 0.74025202 0.74025202 0.74020392        nan        nan
 0.5               nan 0.5        0.74847606 0.74847606 0.74746108
 0.74731606 0.7472176         nan        nan 0.75544363        nan
 0.75544363 0.78295938 0.78295938 0.76644765 0.76896409 0.76664768
        nan        nan 0.73233063        nan 0.7370837  0.79015732
 0.79015732 0.74730439 0.7546479  0.7620011         nan        nan
 0.78532875        nan 0.76479539 0.77261944 0.77261944 0.7239449
 0.75231877 0.76010533        nan        nan 0.75999325        nan
 0.76133006 0.74541741 0.74551361 0.71625363 0.75183333 0.76015341
        nan        nan 0.71731759        nan 0.76025333 0.71965509
 0.71965509 0.70421088 0.7518318  0.76015341        nan        nan
 0.70406089        nan 0.76010533 0.7200558  0.7200557

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73981352
 0.73981352 0.74024635 0.74019825 0.74024635        nan        nan
 0.5               nan 0.5        0.74933393 0.74933393 0.74778384
 0.74759229 0.74749683        nan        nan 0.75544363        nan
 0.75544363 0.78410414 0.78405604 0.76626917 0.77000555 0.76671921
        nan        nan 0.73233063        nan 0.73708373 0.78426911
 0.78426911 0.74467567 0.75738056 0.76308601        nan        nan
 0.78459945        nan 0.76841459 0.76740005 0.76744889 0.72305745
 0.75363551 0.76211199        nan        nan 0.7482394         nan
 0.7638561  0.73835949 0.73835949 0.699756

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.740004
 0.740004   0.74044134 0.74048944 0.74044136        nan        nan
 0.5               nan 0.5        0.74866034 0.74866034 0.74744912
 0.7473063  0.74716129        nan        nan 0.75544363        nan
 0.75544363 0.78715509 0.78715509 0.77097282 0.77388288 0.77102402
        nan        nan 0.73233063        nan 0.73708373 0.7924017
 0.7924017  0.75571211 0.76773421 0.76898853        nan        nan
 0.77514805        nan 0.76931085 0.76897984 0.76897984 0.72551206
 0.76369963 0.76908538        nan        nan 0.74264395        nan
 0.76884375 0.75352157 0.75352157 0.70736911 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73952399
 0.73952399 0.7399124  0.7399124  0.7398643         nan        nan
 0.5               nan 0.5        0.7478481  0.7478481  0.74601093
 0.74610787 0.7456247         nan        nan 0.75544363        nan
 0.75544363 0.78440109 0.78440109 0.76956622 0.77219366 0.76992075
        nan        nan 0.73233063        nan 0.73708373 0.7915614
 0.7915614  0.75570141 0.76441623 0.76687713        nan        nan
 0.77915083        nan 0.77038344 0.75987138 0.75987138 0.71943598
 0.76207986 0.76600154        nan        nan 0.74770045        nan
 0.76793304 0.74184676 0.74179792 0.6961536

In [103]:
print(np.max(best_pca_train_scores))
print(best_pca_test_scores[np.argmax(best_pca_train_scores)])
print("n_components:")
print(n_components[np.argmax(best_pca_train_scores)])

print(best_pca_train_scores)
print(best_pca_test_scores)

0.7679131343630241
0    0.731618
Name: AUC, dtype: float64
n_components:
140
[0    0.735377
Name: AUC, dtype: float64, 0    0.718025
Name: AUC, dtype: float64, 0    0.745921
Name: AUC, dtype: float64, 0    0.750508
Name: AUC, dtype: float64, 0    0.747663
Name: AUC, dtype: float64, 0    0.761797
Name: AUC, dtype: float64, 0    0.750668
Name: AUC, dtype: float64, 0    0.763326
Name: AUC, dtype: float64, 0    0.756944
Name: AUC, dtype: float64, 0    0.746134
Name: AUC, dtype: float64, 0    0.749405
Name: AUC, dtype: float64, 0    0.757103
Name: AUC, dtype: float64, 0    0.744552
Name: AUC, dtype: float64, 0    0.767913
Name: AUC, dtype: float64, 0    0.739911
Name: AUC, dtype: float64, 0    0.732213
Name: AUC, dtype: float64, 0    0.730524
Name: AUC, dtype: float64, 0    0.726043
Name: AUC, dtype: float64, 0    0.729155
Name: AUC, dtype: float64, 0    0.732053
Name: AUC, dtype: float64, 0    0.732266
Name: AUC, dtype: float64, 0    0.73543
Name: AUC, dtype: float64, 0    0.731947
Name: A